In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：AutoML图像分类模型用于批量预测

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_image_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/automl/showcase_automl_image_classification_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库创建图像分类模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)进行批量预测。

### 数据集

这个教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。在这个教程中使用的数据集版本存储在一个公共云存储桶中。训练好的模型可以预测一幅图像是属于五种花朵中的哪一种：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 目标

在本教程中，您将从Python脚本中创建一个AutoML图像分类模型，然后使用Vertex客户端库进行批量预测。您也可以使用`gcloud`命令行工具或在Google Cloud控制台上在线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

使用批量预测和在线预测之间有一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行按需预测，并实时返回结果。

* 批量预测服务：对整个实例集进行排队（批量）预测，并在准备就绪时将结果存储在Cloud Storage存储桶中。

成本

本教程使用 Google Cloud（GCP）的可计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI
价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/)
根据您预计的使用情况生成成本估算。

安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的*google-cloud-storage*库版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动内核

一旦您安装了Vertex客户端库和Google *cloud-storage*，您需要重新启动笔记本内核，以便它能找到这些包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU 运行时

*如果有选择的话，请确保在 GPU 运行时中运行此笔记本。在 Colab 中，选择* **Runtime > Change Runtime Type > GPU**

### 设置您的谷歌云项目

**以下步骤是必需的，无论您的笔记本环境如何。**

1. [选择或创建一个谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得 $300 的免费信用额用于计算/存储成本。

2. [确保已为您的项目启用了计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [谷歌云 SDK](https://cloud.google.com/sdk) 已经安装在谷歌云笔记本中。

5. 在下面的单元格中输入您的项目 ID。然后运行该单元格，以确保 Cloud SDK 对本笔记本中的所有命令使用正确的项目。

**注意**：Jupyter 将以 `!` 为前缀的行视为 shell 命令，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 区域

您还可以更改`REGION`变量，该变量用于笔记本其余部分的操作。以下是Vertex支持的区域。我们建议您选择距离您最近的区域。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不可以使用多区域存储桶来训练Vertex。并非所有区域都支持所有的Vertex服务。有关每个区域的最新支持情况，请参阅[Vertex位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在参加实时教程会话，您可能会使用一个共享的测试帐号或项目。为了避免用户资源创建时的名称冲突，您为每个实例会话创建一个时间戳，并附加到将在本教程中创建的资源名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的谷歌云账号

**如果您正在使用谷歌云笔记本**，则您的环境已经经过身份验证。跳过此步骤。

**如果您正在使用Colab**，请运行下面的单元格，并按提示进行身份验证以通过oAuth验证您的帐户。

**否则**，请按照以下步骤进行操作：

在 Cloud 控制台中，转到 [创建服务账号密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey) 页面。

**点击创建服务账号**。

在**服务账号名称**字段中输入名称，然后点击**创建**。

在**授予此服务账号对项目的访问权限**部分，点击角色下拉列表。在过滤框中输入“Vertex”，并选择**Vertex管理员**。在过滤框中输入“存储对象管理员”，并选择**存储对象管理员**。

点击创建。将包含您密钥的 JSON 文件下载到本地环境。

在下面的单元格中输入您服务账号密钥的路径作为 GOOGLE_APPLICATION_CREDENTIALS 变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

创建一个云存储桶

**无论您使用什么笔记本环境，都需要遵循以下步骤。**

本教程旨在使用位于公共云存储桶以及本地云存储桶的训练数据进行批量预测。您也可以使用存储在本地云存储桶中的自己的训练数据。

在下面设置您的云存储桶的名称。存储桶的名称必须在所有谷歌云项目中全局唯一，包括组织外部的那些云项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有当您的存储桶不存在时，才运行下面的单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查云存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库和定义常量

#### 导入Vertex客户端库

将Vertex客户端库导入我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

顶点常量

为顶点设置以下常量：

- `API_ENDPOINT`：用于数据集、模型、作业、管道和端点服务的顶点 API 服务端点。
- `PARENT`：顶点位置根路径，用于数据集、模型、作业、管道和端点资源。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

自动机器学习常量

设置专属于AutoML数据集和训练的常量：

- 数据集架构：告诉`数据集`资源服务数据集的类型是什么。
- 数据标注（注释）架构：告诉`数据集`资源服务数据如何被标记（注释）。
- 数据集训练架构：告诉`管道`资源服务要为哪种任务（例如分类）训练模型。

In [ ]:
# Image Dataset type
DATA_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
# Image Labeling type
LABEL_SCHEMA = "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_classification_single_label_io_format_1.0.0.yaml"
# Image Training task
TRAINING_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_classification_1.0.0.yaml"

硬件加速器

设置硬件加速器（例如，GPU）用于预测。

设置变量`DEPLOY_GPU/DEPLOY_NGPU`，以使用支持GPU和分配给虚拟机（VM）实例的GPU数量的容器映像。例如，要使用一个具有4个Nvidia Telsa K80 GPU分配给每个VM的GPU容器映像，您可以指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80，4）

对于GPU，可用的加速器包括：
- aip.AcceleratorType.NVIDIA_TESLA_K80
- aip.AcceleratorType.NVIDIA_TESLA_P100
- aip.AcceleratorType.NVIDIA_TESLA_P4
- aip.AcceleratorType.NVIDIA_TESLA_T4
- aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以使用一个容器映像在CPU上运行。

In [ ]:
if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

Docker容器（Docker）镜像

对于AutoML批量预测，顶点预测服务会预先确定用于提供二进制文件的容器镜像。具体来说，根据您选择的硬件加速器，该服务将为模型选择适当的容器。

#### 机器类型

接下来，设置用于预测的机器类型。

-将变量`DEPLOY_COMPUTE`设置为配置用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个vCPU的内存为3.75GB。
     - `n1-highmem`：每个vCPU的内存为6.5GB
     - `n1-highcpu`：每个vCPU的内存为0.9GB
 - `vCPUs`：数量范围为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：您也可以使用n2和e2机器类型进行培训和部署，但它们不支持GPU*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在，您已经准备好开始创建自己的AutoML图像分类模型。

## 设置客户端

Vertex 客户端库以客户端/服务器模型运作。在您的一侧（Python 脚本），您将创建一个客户端，该客户端向 Vertex 服务器发送请求并接收响应。

在本教程中，您将为工作流程中的不同步骤使用不同的客户端。因此，请提前设置它们。

- 数据集服务用于`数据集`资源。
- 模型服务用于`模型`资源。
- 管道服务用于训练。
- 作业服务用于批量预测和自定义训练。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

## 数据集

现在您的客户已经准备好了，训练模型的第一步是创建一个托管的数据集实例，然后将标记的数据上传到其中。

### 创建 `Dataset` 资源实例

使用辅助函数 `create_dataset` 来创建 `Dataset` 资源的实例。这个函数会执行以下操作：

1. 使用数据集客户端服务。
2. 创建一个 Vertex `Dataset` 资源 (`aip.Dataset`)，带有以下参数：
   - `display_name`: 您选择的可读性高的名称。
   - `metadata_schema_uri`: 数据集类型的架构。
3. 调用客户端数据集服务方法 `create_dataset`，带有以下参数：
   - `parent`: 用于您的 `Database`、`Model` 和 `Endpoint` 资源的 Vertex 位置根路径。
   - `dataset`: 您创建的 Vertex 数据集对象实例。
4. 该方法会返回一个 `operation` 对象。

`operation` 对象是 Vertex 处理长时间运行操作的异步调用方式。虽然这一步通常很快，但在项目中首次使用时，由于需要进行配置，可能会有较长延迟。

您可以使用 `operation` 对象来获取操作的状态（例如，创建 `Dataset` 资源），或者取消操作，通过调用操作方法：

| 方法        | 描述          |
| ----------- | --------------|
| result()    | 等待操作完成并返回 JSON 格式的结果对象。 |
| running()   | 返回操作是否仍在运行的 True/False。       |
| done()      | 返回操作是否已完成的 True/False。         |
| canceled()  | 返回操作是否已取消的 True/False。         |
| cancel()    | 取消操作（可能需要多达 30 秒）。          |

In [ ]:
TIMEOUT = 90


def create_dataset(name, schema, labels=None, timeout=TIMEOUT):
    start_time = time.time()
    try:
        dataset = aip.Dataset(
            display_name=name, metadata_schema_uri=schema, labels=labels
        )

        operation = clients["dataset"].create_dataset(parent=PARENT, dataset=dataset)
        print("Long running operation:", operation.operation.name)
        result = operation.result(timeout=TIMEOUT)
        print("time:", time.time() - start_time)
        print("response")
        print(" name:", result.name)
        print(" display_name:", result.display_name)
        print(" metadata_schema_uri:", result.metadata_schema_uri)
        print(" metadata:", dict(result.metadata))
        print(" create_time:", result.create_time)
        print(" update_time:", result.update_time)
        print(" etag:", result.etag)
        print(" labels:", dict(result.labels))
        return result
    except Exception as e:
        print("exception:", e)
        return None


result = create_dataset("flowers-" + TIMESTAMP, DATA_SCHEMA)

现在保存您创建的“数据集”资源实例的唯一数据集标识符。

In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### 数据准备

用于图像的 Vertex `Dataset` 资源对您的数据有一些要求：

- 图像必须存储在 Cloud Storage 存储桶中。
- 每个图像文件必须是图像格式（PNG、JPEG、BMP 等）。
- 在您的 Cloud Storage 存储桶中必须存储一个包含每个图像路径和标签的索引文件。
- 索引文件必须是 CSV 或 JSONL 格式。

CSV索引文件中对于图像分类有以下要求：

- 不包含标题。
- 第一列是图像的云存储路径。
- 第二列是标签。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

快速查看您的数据

您将使用存储在公共云存储桶中的一种花朵数据集版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来统计示例的数量，然后查看前几行数据。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 导入数据

现在，将数据导入到您的Vertex Dataset资源中。使用这个辅助函数`import_data`来导入数据。该函数执行以下操作：

- 使用`Dataset`客户端。
- 调用客户端方法`import_data`，并提供以下参数：
  - `name`：您为`Dataset`资源提供的可读名称（例如，flowers）。
  - `import_configs`：导入配置。

- `import_configs`：包含一个字典的Python列表，其中包含键/值条目：
  - `gcs_sources`：包含一个或多个索引文件路径的URI列表。
  - `import_schema_uri`：标识标签类型的模式。

`import_data()`方法将返回一个长时间运行的`operation`对象。这将需要几分钟才能完成。如果您正在进行实时教程，这将是一个很好的时间来提问或休息一下。

In [ ]:
def import_data(dataset, gcs_sources, schema):
    config = [{"gcs_source": {"uris": gcs_sources}, "import_schema_uri": schema}]
    print("dataset:", dataset_id)
    start_time = time.time()
    try:
        operation = clients["dataset"].import_data(
            name=dataset_id, import_configs=config
        )
        print("Long running operation:", operation.operation.name)

        result = operation.result()
        print("result:", result)
        print("time:", int(time.time() - start_time), "secs")
        print("error:", operation.exception())
        print("meta :", operation.metadata)
        print(
            "after: running:",
            operation.running(),
            "done:",
            operation.done(),
            "cancelled:",
            operation.cancelled(),
        )

        return operation
    except Exception as e:
        print("exception:", e)
        return None


import_data(dataset_id, [IMPORT_FILE], LABEL_SCHEMA)

## 训练模型

现在，使用您的 Vertex `Dataset` 资源来训练一个 AutoML 图像分类模型。要训练模型，请按照以下步骤操作：

1. 为 `Dataset` 资源创建一个 Vertex 训练管道。
2. 执行管道以开始训练。

### 创建一个训练管道

您可能会问，我们用管道做什么？通常在工作（例如训练）有多个步骤时，我们会按顺序执行多个步骤：执行步骤A，执行步骤B等等。通过将这些步骤放入管道中，我们可以获得以下好处：

1. 可重用于后续的训练工作。
2. 可以将其容器化并作为批处理作业运行。
3. 可以进行分布式计算。
4. 所有步骤都与同一管道作业相关联，以跟踪进度。

使用此辅助函数 `create_pipeline`，该函数接受以下参数：

- `pipeline_name`：管道作业的人类可读名称。
- `model_name`：模型的人类可读名称。
- `dataset`：Vertex 完全限定的数据集标识符。
- `schema`：数据集标注训练模式。
- `task`：描述训练作业要求的字典。

该辅助函数调用 `Pipeline` 客户端服务的 `create_pipeline` 方法，该方法接受以下参数：

- `parent`：您的 `数据集`、`模型` 和 `终端` 资源的 Vertex 位置根路径。
- `training_pipeline`：管道训练作业的完整规范。

现在让我们更深入地了解构建 `training_pipeline` 规范的 *最低* 要求：

- `display_name`：管道作业的人类可读名称。
- `training_task_definition`：数据集标注训练模式。
- `training_task_inputs`：描述训练作业要求的字典。
- `model_to_upload`：模型的人类可读名称。
- `input_data_config`：数据集规范。
 - `dataset_id`：仅为 Vertex 数据集标识符（非完全限定）--这是完全限定标识符的最后部分。
 - `fraction_split`：如果指定，数据集用于训练、测试和验证的百分比。否则，百分比会由 AutoML 自动选择。

In [ ]:
def create_pipeline(pipeline_name, model_name, dataset, schema, task):

    dataset_id = dataset.split("/")[-1]

    input_config = {
        "dataset_id": dataset_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    }

    training_pipeline = {
        "display_name": pipeline_name,
        "training_task_definition": schema,
        "training_task_inputs": task,
        "input_data_config": input_config,
        "model_to_upload": {"display_name": model_name},
    }

    try:
        pipeline = clients["pipeline"].create_training_pipeline(
            parent=PARENT, training_pipeline=training_pipeline
        )
        print(pipeline)
    except Exception as e:
        print("exception:", e)
        return None
    return pipeline

### 构建任务需求

接下来，构建任务需求。与其他参数不同，`task`字段接受一个Google protobuf Struct，该结构与Python字典非常相似，而不是接受Python (JSON-like)字典。使用`json_format.ParseDict`方法进行转换。

我们需要指定的最小字段包括：

- `multi_label`：指示是否这是一个多标签 (vs 单一) 分类的True/False值。
- `budget_milli_node_hours`：用于训练模型的最大时间预算（计费时间），其中1000 = 1小时。对于图像分类，预算必须至少为8小时。
- `model_type`：部署模型的类型：
  - `CLOUD`：用于部署到Google Cloud。
  - `MOBILE_TF_LOW_LATENCY_1`：用于部署到边缘并优化延迟（响应时间）。
  - `MOBILE_TF_HIGH_ACCURACY_1`：用于部署到边缘并优化准确性。
  - `MOBILE_TF_VERSATILE_1`：用于部署到边缘并优化延迟和准确性之间的平衡。
- `disable_early_stopping`：指示是否为True/False，让AutoML自行判断是否提前停止训练或者训练完整个预算。

最后，通过调用辅助函数`create_pipeline`创建管道，该函数返回一个训练管道对象的实例。

In [ ]:
PIPE_NAME = "flowers_pipe-" + TIMESTAMP
MODEL_NAME = "flowers_model-" + TIMESTAMP

task = json_format.ParseDict(
    {
        "multi_label": False,
        "budget_milli_node_hours": 8000,
        "model_type": "CLOUD",
        "disable_early_stopping": False,
    },
    Value(),
)

response = create_pipeline(PIPE_NAME, MODEL_NAME, dataset_id, TRAINING_SCHEMA, task)

现在保存您创建的训练管道的唯一标识符。

In [ ]:
# The full unique ID for the pipeline
pipeline_id = response.name
# The short numeric ID for the pipeline
pipeline_short_id = pipeline_id.split("/")[-1]

print(pipeline_id)

### 获取培训管道信息

现在只获取此培训管道实例的管道信息。通过调用作业客户端服务的`get_training_pipeline`方法，使用以下参数获取此作业的作业信息：

- `name`：Vertex完全合格的管道标识符。

当模型培训完成时，管道状态将是`PIPELINE_STATE_SUCCEEDED`。

In [ ]:
def get_training_pipeline(name, silent=False):
    response = clients["pipeline"].get_training_pipeline(name=name)
    if silent:
        return response

    print("pipeline")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" state:", response.state)
    print(" training_task_definition:", response.training_task_definition)
    print(" training_task_inputs:", dict(response.training_task_inputs))
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", dict(response.labels))
    return response


response = get_training_pipeline(pipeline_id)

部署

训练上述模型可能需要超过20分钟的时间。

一旦您的模型训练完成，您可以通过减去`end_time`和`start_time`来计算训练模型所花费的实际时间。对于您的模型，您需要知道完全限定的Vertex模型资源标识符，即管道服务分配的。您可以从返回的管道实例中获取此信息，作为字段`model_to_deploy.name`。

In [ ]:
while True:
    response = get_training_pipeline(pipeline_id, True)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_to_deploy_id = None
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            raise Exception("Training Job Failed")
    else:
        model_to_deploy = response.model_to_upload
        model_to_deploy_id = model_to_deploy.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print("model to deploy:", model_to_deploy_id)

模型信息

现在您的模型已经训练完成，您可以获得关于您的模型的一些信息。

评估模型资源

现在找出模型服务认为你的模型有多好。在训练过程中，数据集的某部分被保留作为测试（留出）数据，由管道服务用于评估模型。

### 列出所有切片的评估

使用这个辅助函数`list_model_evaluations`，它接受以下参数：

- `name`: 用于`Model`资源的Vertex完全合格的模型标识符。

这个辅助函数使用模型客户服务的`list_model_evaluations`方法，该方法接受相同的参数。调用的响应对象是一个列表，其中每个元素都是一个评估指标。

对于每个评估（您可能只有一个），我们然后打印出评估中每个指标的所有键名称，并且对于一个小集合（`logLoss`和`auPrc`）您将打印结果。

In [ ]:
def list_model_evaluations(name):
    response = clients["model"].list_model_evaluations(parent=name)
    for evaluation in response:
        print("model_evaluation")
        print(" name:", evaluation.name)
        print(" metrics_schema_uri:", evaluation.metrics_schema_uri)
        metrics = json_format.MessageToDict(evaluation._pb.metrics)
        for metric in metrics.keys():
            print(metric)
        print("logloss", metrics["logLoss"])
        print("auPrc", metrics["auPrc"])

    return evaluation.name


last_evaluation = list_model_evaluations(model_to_deploy_id)

## 批处理预测的模型部署

现在部署您为批处理预测创建的已训练的 Vertex `Model` 资源。这与为按需预测部署 `Model` 资源不同。

对于在线预测，您需要：

1. 创建一个 `Endpoint` 资源来部署 `Model` 资源。

2. 将 `Model` 资源部署到 `Endpoint` 资源。

3. 向 `Endpoint` 资源发出在线预测请求。

对于批处理预测，您需要：

1. 创建一个批处理预测作业。

2. 作业服务将为批处理预测请求提供资源。

3. 将批处理预测请求的结果返回给调用方。

4. 作业服务将取消为批处理预测请求提供资源。

## 发送批量预测请求

现在对您部署的模型进行批量预测。

获取测试项

现在对您的Vertex模型进行批量预测。您将使用数据集中的任意示例作为测试项。不用担心这些示例很可能在训练模型时使用过--我们只是想演示如何进行预测。

In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n2
if len(str(test_items[0]).split(",")) == 3:
    _, test_item_1, test_label_1 = str(test_items[0]).split(",")
    _, test_item_2, test_label_2 = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1 = str(test_items[0]).split(",")
    test_item_2, test_label_2 = str(test_items[1]).split(",")

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 复制测试项

对于批量预测，您需要将测试项复制到您的云存储桶。

In [ ]:
file_1 = test_item_1.split("/")[-1]
file_2 = test_item_2.split("/")[-1]

! gsutil cp $test_item_1 $BUCKET_NAME/$file_1
! gsutil cp $test_item_2 $BUCKET_NAME/$file_2

test_item_1 = BUCKET_NAME + "/" + file_1
test_item_2 = BUCKET_NAME + "/" + file_2

### 创建批量输入文件

现在制作一个批量输入文件，将其存储在您的本地云存储存储桶中。批量输入文件可以是 CSV 或 JSONL 格式。在本教程中，您将使用 JSONL 格式。对于 JSONL 文件，您需要为每个数据项（实例）的每一行创建一个字典条目。该字典包含以下键值对：

- `content`：图片的云存储路径。
- `mime_type`：内容类型。在我们的示例中，是一个 `jpeg` 文件。

例如：

                        {'content': '[您的存储桶]/file1.jpg', 'mime_type': 'jpeg'}

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_NAME + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": test_item_1, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")
    data = {"content": test_item_2, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 计算实例的扩展

在处理批量预测请求时，您有几种扩展计算实例的选择：

- 单个实例：批量预测请求在单个计算实例上处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动扩展：将批量预测请求分配到您手动指定的固定数量的计算实例上。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数。当模型首次部署到实例时，固定数量的计算实例将被配置，并且批量预测请求将均匀分配在这些实例上。

- 自动扩展：将批量预测请求分配到可扩展数量的计算实例上。
  - 根据负载情况，将计算实例的最小（`MIN_NODES`）数量设置为部署模型时首次配置和取消配置的实例数，并将最大（`MAX_NODES`）数量设置为需要配置的实例数。

计算实例的最小数量对应于字段 `min_replica_count`，最大数量对应于字段 `max_replica_count`，在您后续的部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 发出批量预测请求

现在你的两个测试项目的批量已准备好，让我们进行批量请求。使用以下参数的这个辅助函数 `create_batch_prediction_job`：

- `display_name`：预测作业的可读名称。
- `model_name`：`Model` 资源的 Vertex 完全限定标识符。
- `gcs_source_uri`：输入文件的 Cloud Storage 路径 —— 此文件是您之前创建的。
- `gcs_destination_output_uri_prefix`：服务将写入预测结果的 Cloud Storage 路径。
- `parameters`：用于提供预测结果的附加过滤参数。

辅助函数调用作业客户端服务的 `create_batch_prediction_job` 方法，使用以下参数：

- `parent`：数据集、模型和管道资源的 Vertex 位置根路径。
- `batch_prediction_job`：批量预测作业的规范。

现在让我们深入了解 `batch_prediction_job` 的规范：
- `display_name`：预测批处理作业的可读名称。
- `model`：`Model` 资源的 Vertex 完全限定标识符。
- `dedicated_resources`：为批量预测作业分配的计算资源。
- `model_parameters`：用于提供预测结果的附加过滤参数。
- `input_config`：要预测的实例的输入来源和格式类型。
- `output_config`：预测的输出目的地和格式。

你可能会问，`confidence_threshold` 如何影响模型准确性？阈值不会改变准确性，而是会改变召回率和精确率。

在这个例子中，你将会预测精确度。你将置信阈值设置为 0.5，每个分类的最大预测数为两个。由于各类别的置信值必须加起来为一，只有两种可能的结果：
1. 出现平局，两个值都是 0.5，并返回两个预测结果。
2. 一个值高于 0.5，其他值低于 0.5，并返回一个预测结果。

这次调用是一个异步操作。你将从响应对象中打印出一些选定的字段，包括：
- `name`：分配给批量预测作业的 Vertex 完全限定标识符。
- `display_name`：预测批处理作业的可读名称。
- `model`：Model 资源的 Vertex 完全限定标识符。
- `generate_explanations`：是否提供与预测相关的 True/False 解释（可解释性）。
- `state`：预测作业的状态（挂起、运行等）。

由于这次调用需要一些时间才能执行，你可能会得到 `state` 为 `JobState.JOB_STATE_PENDING`。

In [ ]:
BATCH_MODEL = "flowers_batch-" + TIMESTAMP


def create_batch_prediction_job(
    display_name,
    model_name,
    gcs_source_uri,
    gcs_destination_output_uri_prefix,
    parameters=None,
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    batch_prediction_job = {
        "display_name": display_name,
        # Format: 'projects/{project}/locations/{location}/models/{model_id}'
        "model": model_name,
        "model_parameters": json_format.ParseDict(parameters, Value()),
        "input_config": {
            "instances_format": IN_FORMAT,
            "gcs_source": {"uris": [gcs_source_uri]},
        },
        "output_config": {
            "predictions_format": OUT_FORMAT,
            "gcs_destination": {"output_uri_prefix": gcs_destination_output_uri_prefix},
        },
        "dedicated_resources": {
            "machine_spec": machine_spec,
            "starting_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
        },
    }
    response = clients["job"].create_batch_prediction_job(
        parent=PARENT, batch_prediction_job=batch_prediction_job
    )
    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" create_time:", response.create_time)
    print(" start_time:", response.start_time)
    print(" end_time:", response.end_time)
    print(" update_time:", response.update_time)
    print(" labels:", response.labels)
    return response


IN_FORMAT = "jsonl"
OUT_FORMAT = "jsonl"  # [jsonl]

response = create_batch_prediction_job(
    BATCH_MODEL,
    model_to_deploy_id,
    gcs_input_uri,
    BUCKET_NAME,
    {"confidenceThreshold": 0.5, "maxPredictions": 2},
)

现在获取您创建的批量预测作业的唯一标识符。

In [ ]:
# The full unique ID for the batch job
batch_job_id = response.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### 获取有关批量预测作业的信息

使用此辅助函数`get_batch_prediction_job`，具有以下参数：

- `job_name`：批量预测作业的Vertex完全限定标识符。

辅助函数调用作业客户端服务的`get_batch_prediction_job`方法，具有以下参数：

- `name`：批量预测作业的Vertex完全限定标识符。在本教程中，您将向其传递批量预测作业的Vertex完全限定标识符--`batch_job_id`

辅助函数将返回存储预测结果的Cloud Storage路径--`gcs_destination`。

In [ ]:
def get_batch_prediction_job(job_name, silent=False):
    response = clients["job"].get_batch_prediction_job(name=job_name)
    if silent:
        return response.output_config.gcs_destination.output_uri_prefix, response.state

    print("response")
    print(" name:", response.name)
    print(" display_name:", response.display_name)
    print(" model:", response.model)
    try:  # not all data types support explanations
        print(" generate_explanation:", response.generate_explanation)
    except:
        pass
    print(" state:", response.state)
    print(" error:", response.error)
    gcs_destination = response.output_config.gcs_destination
    print(" gcs_destination")
    print("  output_uri_prefix:", gcs_destination.output_uri_prefix)
    return gcs_destination.output_uri_prefix, response.state


predictions, state = get_batch_prediction_job(batch_job_id)

### 获取预测结果

当批量预测处理完成时，作业状态将为`JOB_STATE_SUCCEEDED`。

最后，您可以查看存储在您设置为输出的Cloud Storage路径中的预测结果。预测结果将以JSONL格式存储，这是您在进行批量预测作业时指定的，存储在以`prediction`起始的子文件夹中，在该文件夹下将有一个名为`predictions*.jsonl`的文件。

现在显示（cat）内容。您将会看到多个JSON对象，每个对象对应一个预测结果。

第一个字段`ID`是您进行预测的图像文件，第二个字段`annotations`是预测结果，进一步分解为：

- `confidences`：置信度（介于0和1之间）。
- `display_name`：对应的类名。

In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    predictions, state = get_batch_prediction_job(batch_job_id, True)
    if state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", state)
        if state == aip.JobState.JOB_STATE_FAILED:
            raise Exception("Batch Job Failed")
    else:
        folder = get_latest_predictions(predictions)
        ! gsutil ls $folder/prediction*.jsonl

        ! gsutil cat $folder/prediction*.jsonl
        break
    time.sleep(60)

清理

要清理此项目中使用的所有GCP资源，您可以删除用于本教程的GCP项目。

否则，您可以删除在本教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME